In [1]:
#importing libraries
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import Dense, Conv1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold
from keras.backend import dropout
import keras
import random
from sklearn.model_selection import train_test_split

2024-08-04 19:29:24.809985: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-04 19:29:25.449737: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-08-04 19:29:25.449804: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-08-04 19:29:25.449809: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
# Set a seed value
seed_value= 3

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)

# 5. For layers that introduce randomness like dropout, make sure to set seed values 
# model.add(Dropout(0.25, seed=seed_value))

In [3]:
#import relevant datasets
X_train = pd.read_csv("finalXtrain.csv")
y_train = pd.read_csv("finalYtrain.csv")
X_test = pd.read_csv("finalXtest.csv")
y_test = pd.read_csv("finalYtest.csv")
X_val = pd.read_csv("finalXval.csv")
y_val = pd.read_csv("finalYval.csv")

In [4]:
#check shape
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

(221, 72)
(221, 3)
(47, 72)
(47, 3)
(48, 72)
(48, 3)


In [5]:
from tensorflow.keras.models import load_model
pre_trained_model = load_model('/home/ridhanya/Desktop/AA_PREDICTION/spring24_new_github/ala_arg_fall_oldfiles/alaarg_oct9_check_works3.h5')

2024-08-04 19:29:26.204708: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-08-04 19:29:26.204758: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (VLSI-CAD): /proc/driver/nvidia/version does not exist
2024-08-04 19:29:26.206071: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
print(pre_trained_model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 72, 1)]           0         
                                                                 
 conv1d (Conv1D)             (None, 24, 5)             20        
                                                                 
 batch_normalization (BatchN  (None, 24, 5)            20        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 24, 5)             0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 5)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 4, 45)             1170  

In [7]:
for layer in pre_trained_model.layers[:-1]:
    print(layer)
    layer.trainable = False

In [8]:
x = pre_trained_model.layers[-2].output 

In [9]:
from tensorflow.keras.models import Model

# new_output = Dense(3, activation='softmax', name="newout")(x)
# x = Dropout(0.5,seed=seed_value)(x)
# x = Dense(20,activation ='sigmoid')(x)
# x = Dense(10,activation ='sigmoid')(x)
# x = Dense(5,activation ='tanh')(x)
outputs = Dense(3, activation='softmax', name='predictions')(x)
model = Model(inputs=pre_trained_model.input, outputs=outputs)

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 72, 1)]           0         
                                                                 
 conv1d (Conv1D)             (None, 24, 5)             20        
                                                                 
 batch_normalization (BatchN  (None, 24, 5)            20        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 24, 5)             0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 5)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 4, 45)             1170  

In [12]:
#earlystopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=50)
checkpoint_path = 'alaarglys_transferfinallayer_model3.h5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                   monitor='val_accuracy',
                                   save_best_only=True,
                                   mode='max',
                                   verbose=1)

In [13]:

history = model.fit(X_train, y_train,
              batch_size=10,
              verbose=1,
              epochs=500,
              validation_data = (X_val,y_val), 
              callbacks = [es,model_checkpoint]
                    ,class_weight = {0:2.4,1:2.23,2:7.37}
                   )

Epoch 1/500
 1/23 [>.............................] - ETA: 10s - loss: 6.8087 - accuracy: 0.1000
Epoch 1: val_accuracy improved from -inf to 0.16667, saving model to alaarglys_transferfinallayer_model3.h5
23/23 [==============================] - 1s 9ms/step - loss: 5.0202 - accuracy: 0.1448 - val_loss: 1.6575 - val_accuracy: 0.1667
Epoch 2/500
 1/23 [>.............................] - ETA: 0s - loss: 4.7403 - accuracy: 0.3000
Epoch 2: val_accuracy improved from 0.16667 to 0.20833, saving model to alaarglys_transferfinallayer_model3.h5
23/23 [==============================] - 0s 3ms/step - loss: 4.6034 - accuracy: 0.1991 - val_loss: 1.5348 - val_accuracy: 0.2083
Epoch 3/500
 1/23 [>.............................] - ETA: 0s - loss: 5.2839 - accuracy: 0.1000
Epoch 3: val_accuracy improved from 0.20833 to 0.22917, saving model to alaarglys_transferfinallayer_model3.h5
23/23 [==============================] - 0s 3ms/step - loss: 4.5853 - accuracy: 0.1900 - val_loss: 1.4513 - val_accuracy: 0.22

 1/23 [>.............................] - ETA: 0s - loss: 3.3967 - accuracy: 0.3000
Epoch 29: val_accuracy did not improve from 0.31250
23/23 [==============================] - 0s 2ms/step - loss: 3.2336 - accuracy: 0.3756 - val_loss: 1.1535 - val_accuracy: 0.2708
Epoch 30/500
 1/23 [>.............................] - ETA: 0s - loss: 4.2774 - accuracy: 0.2000
Epoch 30: val_accuracy did not improve from 0.31250
23/23 [==============================] - 0s 2ms/step - loss: 3.4177 - accuracy: 0.3982 - val_loss: 1.1529 - val_accuracy: 0.2708
Epoch 31/500
 1/23 [>.............................] - ETA: 0s - loss: 3.6404 - accuracy: 0.4000
Epoch 31: val_accuracy did not improve from 0.31250
23/23 [==============================] - 0s 2ms/step - loss: 3.1709 - accuracy: 0.4796 - val_loss: 1.1352 - val_accuracy: 0.2917
Epoch 32/500
 1/23 [>.............................] - ETA: 0s - loss: 4.5751 - accuracy: 0.2000
Epoch 32: val_accuracy did not improve from 0.31250
23/23 [===========================

 1/23 [>.............................] - ETA: 0s - loss: 2.6784 - accuracy: 0.7000
Epoch 57: val_accuracy improved from 0.52083 to 0.54167, saving model to alaarglys_transferfinallayer_model3.h5
23/23 [==============================] - 0s 3ms/step - loss: 3.0458 - accuracy: 0.5747 - val_loss: 1.0804 - val_accuracy: 0.5417
Epoch 58/500
 1/23 [>.............................] - ETA: 0s - loss: 4.6022 - accuracy: 0.2000
Epoch 58: val_accuracy did not improve from 0.54167
23/23 [==============================] - 0s 2ms/step - loss: 3.1075 - accuracy: 0.5204 - val_loss: 1.0834 - val_accuracy: 0.5208
Epoch 59/500
 1/23 [>.............................] - ETA: 0s - loss: 3.7242 - accuracy: 0.7000
Epoch 59: val_accuracy did not improve from 0.54167
23/23 [==============================] - 0s 2ms/step - loss: 2.9850 - accuracy: 0.5385 - val_loss: 1.0970 - val_accuracy: 0.4792
Epoch 60/500
 1/23 [>.............................] - ETA: 0s - loss: 3.3397 - accuracy: 0.6000
Epoch 60: val_accuracy did

23/23 [==============================] - 0s 2ms/step - loss: 2.9786 - accuracy: 0.5656 - val_loss: 1.0850 - val_accuracy: 0.5208
Epoch 87/500
 1/23 [>.............................] - ETA: 0s - loss: 1.9630 - accuracy: 0.8000
Epoch 87: val_accuracy did not improve from 0.54167
23/23 [==============================] - 0s 2ms/step - loss: 2.8991 - accuracy: 0.5792 - val_loss: 1.0817 - val_accuracy: 0.5417
Epoch 88/500
 1/23 [>.............................] - ETA: 0s - loss: 2.2895 - accuracy: 0.3000
Epoch 88: val_accuracy did not improve from 0.54167
23/23 [==============================] - 0s 2ms/step - loss: 2.7890 - accuracy: 0.6154 - val_loss: 1.0837 - val_accuracy: 0.5417
Epoch 89/500
 1/23 [>.............................] - ETA: 0s - loss: 2.7692 - accuracy: 0.3000
Epoch 89: val_accuracy did not improve from 0.54167
23/23 [==============================] - 0s 2ms/step - loss: 2.9791 - accuracy: 0.5701 - val_loss: 1.0863 - val_accuracy: 0.5417
Epoch 90/500
 1/23 [>...................

In [14]:
model1 = tf.keras.models.load_model('alaarglys_transferfinallayer_model3.h5')
model1.evaluate(X_test,y_test)

2/2 [==============================] - 0s 2ms/step - loss: 0.9486 - accuracy: 0.5957


[0.9485639929771423, 0.5957446694374084]

In [15]:
y=model1.predict(X_test)

2/2 [==============================] - 0s 2ms/step


In [16]:
#class-wise accuracy
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test.to_numpy().argmax(axis=1), y.argmax(axis=1))
matrix.diagonal()/matrix.sum(axis=1)

array([0.6       , 0.71428571, 0.16666667])

In [17]:
y_test[:2]

,L_Alanine,L_Arginine,L_Lysine
0,0,1,0
1,0,1,0


In [18]:
model1.evaluate(X_val,y_val)

2/2 [==============================] - 0s 2ms/step - loss: 1.0804 - accuracy: 0.5417


[1.08035147190094, 0.5416666865348816]

In [19]:
model1.evaluate(X_train,y_train)

7/7 [==============================] - 0s 1ms/step - loss: 0.9116 - accuracy: 0.6109


[0.9116246104240417, 0.610859751701355]